# Limpieza datos educativos

In [1]:
import pandas as pd
import os
import re

In [2]:
# Cargar el CSV
df = pd.read_csv('republica.csv')
print(f"Cantidad de registros: {df.size}")

Cantidad de registros: 112319


# Filtro
Enfocado en los valores de establecimiento

In [3]:
# Función para limpiar nombres
def normalize_name(name):
    if pd.isnull(name):
        return None  # Cambia "" por None para que se pueda filtrar luego
    name = str(name).upper()
    name = re.sub(r'[\s\-]+', ' ', name)  # reemplaza múltiples espacios o guiones por un solo espacio
    name = name.strip()
    name = re.sub(r'[^\x00-\x7F]+', '', name)  # elimina caracteres no ASCII (invisibles)
    name = name.replace('"', ' ').strip()
    name = name.replace('  ', ' ').strip().replace(' -', '-').replace('- ', '-')
    return name

# Aplicar limpieza
df["ESTABLECIMIENTO_CLEAN"] = df["ESTABLECIMIENTO"].apply(normalize_name)
df["ESTABLECIMIENTO_VARIANTES"] = df["ESTABLECIMIENTO"]

# Eliminar filas donde el nombre limpio es None
df = df[df["ESTABLECIMIENTO_CLEAN"].notna()]

# Agrupar por nombre limpio y juntar resto en sets
def group_sets(col):
    return list(set(col.dropna()))

grouped1 = df.groupby("ESTABLECIMIENTO_CLEAN").agg(group_sets).reset_index()
grouped1 = grouped1.drop(columns=["ESTABLECIMIENTO"])

# Guardar resultado
grouped1.to_csv('limpio.csv', index=False)

In [4]:
print(f"Cantidad de registros: {grouped1.size}")

Cantidad de registros: 63558


## Filtro 2
Revisión de caracteres extra

In [5]:
def group_strings(col):
    flattened = []
    for item in col.dropna():
        # Si el item es string que parece una lista, intenta parsearlo
        if isinstance(item, str) and item.startswith("[") and item.endswith("]"):
            try:
                import ast
                parsed = ast.literal_eval(item)
                if isinstance(parsed, list):
                    flattened.extend(parsed)
                else:
                    flattened.append(str(parsed))
            except:
                flattened.append(str(item))
        # Si ya es lista, extiende
        elif isinstance(item, list):
            flattened.extend(item)
        else:
            flattened.append(str(item))

    # Convertir todo a string y eliminar duplicados
    return ", ".join(sorted(set(str(i).strip() for i in flattened)))

In [6]:
from rapidfuzz import fuzz, process

df = pd.read_csv('limpio.csv')

# Agrupación fuzzy
unique_names = df["ESTABLECIMIENTO_CLEAN"].unique()
name_map = {}
visited = set()

for name in unique_names:
    if name in visited:
        continue
    # Buscar similares (umbral de similitud >= 90)
    matches = process.extract(name, unique_names, scorer=fuzz.token_sort_ratio, limit=None)
    group = [match for match, score, _ in matches if score >= 90]
    for g in group:
        name_map[g] = name  # agrupar bajo el nombre base
        visited.add(g)

# Reemplazar por nombre base en el DataFrame
df["ESTABLECIMIENTO"] = df["ESTABLECIMIENTO_CLEAN"].map(name_map)

grouped = df.groupby("ESTABLECIMIENTO").agg(group_strings).reset_index()

# Eliminar columnas que ya no necesitas
grouped = grouped.drop(columns=["ESTABLECIMIENTO_CLEAN"])

# Guardar
grouped.to_csv("establecimientos_fuzzy.csv", index=False)


In [7]:
print(f"Cantidad de registros: {grouped.size}")

Cantidad de registros: 51408


## Filtro 3

Eliminacion de Duplicados

In [5]:
import pandas as pd
from fuzzywuzzy import fuzz
from itertools import combinations

# Cargar el archivo
df = pd.read_csv("establecimientos_fuzzy.csv")

# Limpieza básica
df['ESTABLECIMIENTO'] = df['ESTABLECIMIENTO'].str.strip("'").str.strip('"')
df['DISTRITO'] = df['DISTRITO'].astype(str).str.strip()
df['DEPARTAMENTO'] = df['DEPARTAMENTO'].astype(str).str.strip()

# Lista para resultados
posibles_duplicados = []

# Agrupar por departamento y distrito para limitar las comparaciones
for (dep, dist), grupo in df.groupby(['DEPARTAMENTO', 'DISTRITO']):
    for (idx1, row1), (idx2, row2) in combinations(grupo.iterrows(), 2):
        nombre1 = row1['ESTABLECIMIENTO']
        nombre2 = row2['ESTABLECIMIENTO']
        
        # Fuzzy matching
        similarity = fuzz.token_sort_ratio(nombre1, nombre2)
        
        if similarity >= 90 and nombre1 != nombre2:
            posibles_duplicados.append({
                'departamento': dep,
                'distrito': dist,
                'idx_1': idx1,
                'idx_2': idx2,
                'nombre_1': nombre1,
                'nombre_2': nombre2,
                'direccion_1': row1['DIRECCION'],
                'direccion_2': row2['DIRECCION'],
                'similitud': similarity
            })

# Guardar como CSV
duplicados_df = pd.DataFrame(posibles_duplicados)
duplicados_df.to_csv("duplicados_fuzzy_localizados.csv", index=False)

print("Duplicados fuzzy por ubicación guardados en 'duplicados_fuzzy_localizados.csv'")



Duplicados fuzzy por ubicación guardados en 'duplicados_fuzzy_localizados.csv'


In [6]:
import pandas as pd
from fuzzywuzzy import fuzz

# Cargar los datos (ejemplo)
df = pd.read_csv('duplicados_fuzzy_localizados.csv')

# Normalizar direcciones: mayúsculas y quitar signos comunes
def normalizar_dir(d):
    return d.upper().replace('.', '').replace(',', '').strip()

df['dir1_norm'] = df['direccion_1'].apply(normalizar_dir)
df['dir2_norm'] = df['direccion_2'].apply(normalizar_dir)

# Calcular similitud difusa entre direcciones
df['sim_dir'] = df.apply(lambda x: fuzz.ratio(x['dir1_norm'], x['dir2_norm']), axis=1)

# Filtrar duplicados exactos o muy cercanos
duplicados_mismos_lugar = df[df['sim_dir'] >= 95]

# Sucursales probables
sucursales = df[(df['sim_dir'] < 95) & (df['similitud'] >= 85)]

print("Posibles duplicados mismos lugares:\n", duplicados_mismos_lugar[['nombre_1', 'nombre_2', 'direccion_1', 'direccion_2', 'sim_dir']])
print("\nPosibles sucursales o diferentes sedes:\n", sucursales[['nombre_1', 'nombre_2', 'direccion_1', 'direccion_2', 'sim_dir']])


Posibles duplicados mismos lugares:
                                              nombre_1  \
0   CENTRO DE ESTUDIOS TCNICOS Y AVANZADOS DE CHIM...   
5   INSTITUTO DE EDUCACION DIVERSIFICADA 'RAFAEL L...   
7               COLEGIO MIXTO 'CRISTIANO GUATEMALTECO   
8   INSTITUTO GUATEMALTECO DE EDUCACIN RADIOFNICA ...   
13       LICEO CRISTIANO GUATEMALTECO 'NUEVO AMANECER   
15         COLEGIO MIXTO CRISTIANO 'NUEVOS HORIZONTES   
17  COLEGIO DE 'ESTUDIOS TECNICOS ESPECIALIZADOS E...   

                                             nombre_2  \
0   CENTRO DE ESTUDIOS TCNICOS Y AVANZADOS DE CHIM...   
5   INSTITUTO DE EDUCACION DIVERSIFICADA RAFAEL LA...   
7                COLEGIO MIXTO CRISTIANO GUATEMALTECO   
8   INSTITUTO GUATEMALTECO DE EDUCACION RADIOFONIC...   
13        LICEO CRISTIANO GUATEMALTECO NUEVO AMANECER   
15          COLEGIO MIXTO CRISTIANO NUEVOS HORIZONTES   
17  COLEGIO DE ESTUDIOS TECNICOS ESPECIALIZADOS EN...   

                                        direccion

In [10]:
import re 

def normalizar_texto(texto):
    if pd.isna(texto):
        return ''
    return re.sub(r'[^\w\s]', '', texto).strip().upper()

# Carga datasets
df_colegios = pd.read_csv('establecimientos_fuzzy.csv')  # dataset con toda la info original
df_duplicados = pd.read_csv('duplicados_fuzzy_localizados.csv')  # dataset con pares duplicados y similitudes

# Normalizar direcciones para fuzzy matching
df_duplicados['dir1_norm'] = df_duplicados['direccion_1'].apply(normalizar_texto)
df_duplicados['dir2_norm'] = df_duplicados['direccion_2'].apply(normalizar_texto)

# Calcular similitud entre direcciones
df_duplicados['sim_dir'] = df_duplicados.apply(lambda x: fuzz.ratio(x['dir1_norm'], x['dir2_norm']), axis=1)

# Separar grupos
duplicados_mismos_lugar = df_duplicados[df_duplicados['sim_dir'] >= 95]
sucursales = df_duplicados[(df_duplicados['sim_dir'] < 95) & (df_duplicados['similitud'] >= 85)]

# --- PROCESAR DUPLICADOS MISMOS LUGARES ---
for _, row in duplicados_mismos_lugar.iterrows():
    nom1 = row['nombre_1']
    nom2 = row['nombre_2']

    # Buscar en df_colegios filas que coincidan con alguno de los dos nombres (normalizados)
    mask1 = df_colegios['ESTABLECIMIENTO'].str.upper().str.contains(normalizar_texto(nom1), na=False)
    mask2 = df_colegios['ESTABLECIMIENTO'].str.upper().str.contains(normalizar_texto(nom2), na=False)

    idx1 = df_colegios[mask1].index
    idx2 = df_colegios[mask2].index

    # Si ambas existen, elegimos cuál mantener (ejemplo: el que tenga nombre más "limpio")
    if len(idx1) > 0 and len(idx2) > 0:
        # Función simple para medir "limpieza" del nombre
        def puntuacion_nombre(n):
            # Menos caracteres especiales, mejor
            return len(re.findall(r'[^\w\s]', n))

        punt1 = puntuacion_nombre(df_colegios.loc[idx1[0], 'ESTABLECIMIENTO'])
        punt2 = puntuacion_nombre(df_colegios.loc[idx2[0], 'ESTABLECIMIENTO'])

        # Mantener el que tenga menos puntuacion (menos caracteres raros)
        if punt1 <= punt2:
            df_colegios = df_colegios.drop(idx2)
        else:
            df_colegios = df_colegios.drop(idx1)

# --- PROCESAR SUCURSALES ---
for _, row in sucursales.iterrows():
    nom1 = row['nombre_1']
    nom2 = row['nombre_2']
    dir1 = row['direccion_1']
    dir2 = row['direccion_2']

    # Buscar índice de la fila principal (usaremos nom1 para buscar)
    mask = df_colegios['ESTABLECIMIENTO'].str.upper().str.contains(normalizar_texto(nom1), na=False)
    idx = df_colegios[mask].index

    if len(idx) > 0:
        idx = idx[0]
        # Agregar las variantes: nombre2 y dirección 2 concatenados
        variantes_actual = df_colegios.at[idx, 'ESTABLECIMIENTO_VARIANTES']
        if pd.isna(variantes_actual):
            variantes_actual = ''

        # Agregar variantes en formato "Nombre - Dirección"
        nueva_variante = f"{nom2} - {dir2}"
        if nueva_variante.upper() not in variantes_actual.upper():
            if variantes_actual != '':
                variantes_actual += ', '
            variantes_actual += nueva_variante

        df_colegios.at[idx, 'ESTABLECIMIENTO_VARIANTES'] = variantes_actual

# Guardar el dataset actualizado
df_colegios.to_csv('dataset_fuzzy_no_duplicados.csv', index=False)